In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import math
import csv
import os

## ILP for the Unbounded Multiple Knapsack Problem with Fixed Processing Times

In [2]:
n = 10 # number of campaigns
m = 1000 # number of customers
slots = 3 # 3 time slots
slot_length = 15

s = np.random.choice([0, 1, 2], n) # slot
p = np.full(n, slot_length) # processing time
c = np.random.randint(low = 10, high = 40, size = n) # cost
b = np.random.randint(low = 40, high = 80, size = n) # benefit
C = np.random.randint(low = 40, high = 100, size = n) # maximum cost

zippedList =  list(zip(s, p, c, b, C))
campaigns = pd.DataFrame(zippedList, columns = ['starting time', 'processing time' , 'cost', 'benefit', 'maximum cost'])
campaigns.index.names = ['name']
# print("Dataframe : " , campaigns, sep='\n')

# matrix with take-rates
T = np.random.random_sample((m, n))

In [32]:
# create empty model
model = gp.Model()

# add decision variables
x = model.addMVar((m,n), vtype=GRB.BINARY, name='x') # variable x_ij
z = model.addMVar((m,n,slots), vtype=GRB.BINARY, name = 'z') # variable z_ijl
 
# set objective function
model.setObjective((sum((b[j]*T[i][j] - c[j]) * x[i,j] for j in range(n) for i in range(m))), GRB.MAXIMIZE)

# add constraints
model.addConstrs(((sum(c[j]*x[i,j] for i in range(m))) <= C[j]) for j in range(n))

model.addConstrs(((sum(z[i,j,l] for j in range(n))) <= 1) for i in range(m) for l in range(slots))

model.addConstrs((z[i,j,l] == 0) for j in range(n) for i in range(m) for l in range(slots) if l != s[j])

# solve model
model.optimize()

# get attributes
if model.SolCount > 0:
    print(model.objVal)
    x_values = []
    for k in range(m*n):
        current = model.getVarByName("x[{}]".format(k))
        x_values.append((current.varName, current.x))

cwd = os.getcwd()
path = '{}/output'.format(cwd)
if not os.path.exists(path):
    os.makedirs(path)
    
# write schedule into file
with open('{}/fptumkp.csv'.format(path), mode = 'w') as schedule:
    writer = csv.writer(schedule, delimiter = ';')
    writer.writerow(['campaign','customer','starting time', 'processing time'])

    for value in x_values:
        if value[1] == 1.0:
            count = int(value[0][value[0].find('[')+1:value[0].find(']')])
            customer = math.floor(count/n)
            campaign = count%n
            print(campaign, customer, s[campaign], p[campaign])

schedule.close()

# export model
# model.write('fptumkp.lp')

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 23010 rows, 40000 columns and 60000 nonzeros
Model fingerprint: 0x787b454f
Variable types: 0 continuous, 40000 integer (40000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [5e-03, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
Found heuristic solution: objective 675.4610657
Presolve removed 23009 rows and 39151 columns
Presolve time: 0.20s
Presolved: 1 rows, 849 columns, 849 nonzeros
Found heuristic solution: objective 1305.9400996
Variable types: 0 continuous, 849 integer (849 binary)

Root relaxation: objective 1.306886e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1306.8860105 1306.88601  0.00%  